### import packages

In [151]:
import pandas as pd
import numpy as np



import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 1. Download and Explore Dataset

### Pull data from Web page using Pandas library

In [152]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_list = pd.read_html(url)

#### Check content of dataframe

In [153]:
df_list

[    Postal Code           Borough  \
 0           M1A      Not assigned   
 1           M2A      Not assigned   
 2           M3A        North York   
 3           M4A        North York   
 4           M5A  Downtown Toronto   
 ..          ...               ...   
 175         M5Z      Not assigned   
 176         M6Z      Not assigned   
 177         M7Z      Not assigned   
 178         M8Z         Etobicoke   
 179         M9Z      Not assigned   
 
                                          Neighbourhood  
 0                                         Not assigned  
 1                                         Not assigned  
 2                                            Parkwoods  
 3                                     Victoria Village  
 4                            Regent Park, Harbourfront  
 ..                                                 ...  
 175                                       Not assigned  
 176                                       Not assigned  
 177                

#### We have 3 different dataframes which has been pulled from webpage

In [154]:
len(df_list)

3

#### We are interested on 1st data frame as this contains required info for this assignment

In [155]:
df_list[0]

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


#### Saving / Backup of original dataframe 

Created new dataframe and named as **df**

In [156]:
df = df_list[0]

In [157]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


##### Check type of dataframe

In [158]:
type(df)

pandas.core.frame.DataFrame

##### Check shape of the dataframe

In [160]:
df.shape

(180, 3)

###### Drop rows from data frame where Borough vlaue is Not Assigned

In [161]:
# Get names of indexes for which column Age has value 30
indexNames = df[ df['Borough'] == 'Not assigned' ].index

# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)

df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Check shape of dataframe

In [162]:
df.shape

(103, 3)

##### More than one neighborhood can exist in one postal code area

These will be comma seperated values in Neighbourhood columns

In [163]:
df[ df['Postal Code'] == 'M5A' ]

,Postal Code,Borough,Neighbourhood
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


##### number of rows of your dataframe

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [164]:
df.shape

(103, 3)

### Create Geospatial dataframe with lat lon values

#### Download Geospatial data csv file

In [165]:
import urllib.request
url = 'https://cocl.us/Geospatial_data'
filename = 'Geospatial_Coordinates.csv'
urllib.request.urlretrieve(url, filename)

('Geospatial_Coordinates.csv', <http.client.HTTPMessage at 0x218257ce348>)

#### Read CSV records into a dataframe

In [166]:
df1 = pd.read_csv('Geospatial_Coordinates.csv')
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Check the shape ( row and columns) in the dataframe

In [167]:
df1.shape

(103, 3)

##### Data Preprocessing 

Merge both dataframes to get a new data frame with PostalCode , Borough , Neighbourhood ,Lat and Lon vlaues

In [168]:
df2 = pd.merge(df, df1, on='Postal Code')

In [169]:
df2.head(11)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [170]:
df2.shape

(103, 5)

## Explore and cluster the neighborhoods in Toronto

In [171]:
# import remaning pacakges


import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [172]:


toronto_data = df2[df2['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [173]:
toronto_data.shape

(39, 5)

#### Use geopy library to get the latitude and longitude values of Toronto.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.


In [174]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.


In [175]:
# create map of Toronto using latitude and longitude values

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## 2. Explore Neighborhoods in Toronto

#### Define Foursquare Credentials and Version

In [176]:
CLIENT_ID = '3HQYVG0MTRCTGXWRFDH535ADLCESDR1SRWM2MHEXGRDJKABO' # your Foursquare ID
CLIENT_SECRET = 'JOBKGZRYU3W10T53NIWJZ0TKVYGRZFRFTSDAYIGKJZEKLNNX' # your Foursquare Secret
VERSION = '20201126' # Foursquare API version
LIMIT = 50 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3HQYVG0MTRCTGXWRFDH535ADLCESDR1SRWM2MHEXGRDJKABO
CLIENT_SECRET:JOBKGZRYU3W10T53NIWJZ0TKVYGRZFRFTSDAYIGKJZEKLNNX


#### Let's create a function to repeat the same process to all the neighborhoods in Toronto


In [178]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.


In [179]:
# type your answer here
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )
toronto_venues.shape

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

(1186, 7)

In [180]:
print(toronto_venues.shape)
toronto_venues.head()

(1186, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


Let's check how many venues were returned for each neighborhood

In [181]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,50,50,50,50,50,50
"Brockton, Parkdale Village, Exhibition Place",25,25,25,25,25,25
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,50,50,50,50,50,50
Christie,16,16,16,16,16,16
Church and Wellesley,50,50,50,50,50,50
"Commerce Court, Victoria Hotel",50,50,50,50,50,50
Davisville,34,34,34,34,34,34


#### Let's find out how many unique categories can be curated from all the returned venues


In [182]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 217 uniques categories.


##  3. Analyze Each Neighborhood


In [183]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])

toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [184]:
toronto_onehot.shape

(1186, 217)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [185]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.02,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.00
1,"Brockton, Parkdale Village, Exhibition Place",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
2,"Business reply mail Processing Centre, South C...",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0.058824,0.058824,0.058824,0.117647,0.117647,0.058824,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
4,Central Bay Street,0.02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.02,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.02


#### Let's confirm the new size


In [187]:
toronto_grouped.shape

(39, 217)

#### Let's print each neighborhood along with the top 5 most common venues


In [188]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1      Farmers Market  0.04
2  Seafood Restaurant  0.04
3              Bakery  0.04
4          Restaurant  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.12
1     Coffee Shop  0.08
2       Nightclub  0.08
3  Breakfast Spot  0.08
4             Gym  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0          Restaurant  0.06
1    Recording Studio  0.06
2          Comic Shop  0.06
3       Burrito Place  0.06
4  Light Rail Station  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
             venue  freq
0   Airport Lounge  0.12
1  Airport Service  0.12
2         Boutique  0.06
3          Airport  0.06
4            Plane  0.06


----Central Bay Street----
                    venue  freq
0             Coffee S

#### Let's put that into a _pandas_ dataframe


First, let's write a function to sort the venues in descending order.


In [189]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [190]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Beer Bar,Cocktail Bar,Seafood Restaurant,Cheese Shop,Restaurant,Farmers Market,Japanese Restaurant,Beach
1,"Brockton, Parkdale Village, Exhibition Place",Café,Nightclub,Coffee Shop,Breakfast Spot,Performing Arts Venue,Bakery,Stadium,Restaurant,Intersection,Italian Restaurant
2,"Business reply mail Processing Centre, South C...",Garden,Gym / Fitness Center,Light Rail Station,Brewery,Burrito Place,Restaurant,Butcher,Recording Studio,Farmers Market,Fast Food Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Boat or Ferry,Coffee Shop,Boutique,Rental Car Location,Bar,Historic Site,Harbor / Marina,Plane
4,Central Bay Street,Coffee Shop,Italian Restaurant,Burger Joint,Café,New American Restaurant,Japanese Restaurant,Juice Bar,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant


## 4. Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.


In [191]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [192]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with tornoto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Yoga Studio,Shoe Store,Brewery
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,3,Coffee Shop,Yoga Studio,Bank,Smoothie Shop,Beer Bar,Sandwich Place,Restaurant,Café,Portuguese Restaurant,Chinese Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Coffee Shop,Clothing Store,Café,Bookstore,Ramen Restaurant,Italian Restaurant,Theater,Fast Food Restaurant,Pizza Place,Sandwich Place
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Café,Coffee Shop,Gastropub,Hotel,Japanese Restaurant,Restaurant,Farmers Market,Cocktail Bar,Cosmetics Shop,Beer Bar
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Trail,Pub,Health Food Store,Wine Bar,Cuban Restaurant,Dog Run,Dive Bar,Distribution Center,Discount Store,Diner


Finally, let's visualize the resulting clusters


In [193]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters


#### Cluster 1


In [194]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,0,Park,Swim School,Bus Line,College Gym,Cupcake Shop,Doner Restaurant,Dog Run,Dive Bar,Distribution Center,Discount Store
21,Central Toronto,0,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Park,Furniture / Home Store,Creperie,Distribution Center,Discount Store,Diner
33,Downtown Toronto,0,Park,Trail,Playground,Tennis Court,Creperie,Dog Run,Dive Bar,Distribution Center,Discount Store,Diner


#### Cluster 2


In [195]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,1,Playground,Trail,Wine Bar,Cuban Restaurant,Doner Restaurant,Dog Run,Dive Bar,Distribution Center,Discount Store,Diner


#### Cluster 3

In [196]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,2,Dive Bar,Garden,Music Venue,Wine Bar,Cuban Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop


#### Cluster 4

In [197]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,3,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Yoga Studio,Shoe Store,Brewery
1,Downtown Toronto,3,Coffee Shop,Yoga Studio,Bank,Smoothie Shop,Beer Bar,Sandwich Place,Restaurant,Café,Portuguese Restaurant,Chinese Restaurant
2,Downtown Toronto,3,Coffee Shop,Clothing Store,Café,Bookstore,Ramen Restaurant,Italian Restaurant,Theater,Fast Food Restaurant,Pizza Place,Sandwich Place
3,Downtown Toronto,3,Café,Coffee Shop,Gastropub,Hotel,Japanese Restaurant,Restaurant,Farmers Market,Cocktail Bar,Cosmetics Shop,Beer Bar
5,Downtown Toronto,3,Coffee Shop,Bakery,Beer Bar,Cocktail Bar,Seafood Restaurant,Cheese Shop,Restaurant,Farmers Market,Japanese Restaurant,Beach
6,Downtown Toronto,3,Coffee Shop,Italian Restaurant,Burger Joint,Café,New American Restaurant,Japanese Restaurant,Juice Bar,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant
7,Downtown Toronto,3,Grocery Store,Café,Park,Italian Restaurant,Restaurant,Baby Store,Candy Store,Athletics & Sports,Nightclub,Coffee Shop
8,Downtown Toronto,3,Café,Coffee Shop,Restaurant,Steakhouse,American Restaurant,Concert Hall,Pizza Place,Colombian Restaurant,Gym,Gym / Fitness Center
9,West Toronto,3,Bakery,Pharmacy,Grocery Store,Brewery,Supermarket,Bank,Bar,Middle Eastern Restaurant,Café,Music Venue
10,Downtown Toronto,3,Coffee Shop,Aquarium,Scenic Lookout,Hotel,Plaza,Café,Park,Bar,Dessert Shop,Baseball Stadium


#### Cluster 5


In [198]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,4,Trail,Pub,Health Food Store,Wine Bar,Cuban Restaurant,Dog Run,Dive Bar,Distribution Center,Discount Store,Diner
